In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.initializers import RandomNormal, GlorotNormal
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras import regularizers

def vgg16(act):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = vgg16('tanh')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)


print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

170508288/170498071 [==============================] - 12s 0us/step
Epoch 1/50
313/313 [==============================] - 108s 177ms/step - loss: 5.6250 - accuracy: 0.0992 - val_loss: 4.5504 - val_accuracy: 0.0976
Epoch 2/50
313/313 [==============================] - 51s 164ms/step - loss: 4.9220 - accuracy: 0.0996 - val_loss: 4.4967 - val_accuracy: 0.0973
Epoch 3/50
313/313 [==============================] - 51s 164ms/step - loss: 4.6730 - accuracy: 0.0980 - val_loss: 4.4735 - val_accuracy: 0.1019
Epoch 4/50
313/313 [==============================] - 51s 164ms/step - loss: 4.5631 - accuracy: 0.0998 - val_loss: 4.4573 - val_accuracy: 0.0974
Epoch 5/50
313/313 [==============================] - 51s 164ms/step - loss: 4.5031 - accuracy: 0.1012 - val_loss: 4.4438 - val_accuracy: 0.1013
Epoch 6/50
313/313 [==============================] - 51s 164ms/step - loss: 4.4720 - accuracy: 0.0960 - val_loss: 4.4278 - val_accuracy: 0.1003
Epoch 7/50
313/313 [==============================] - 51s 164